In [23]:
path = "abfss://4155d96d-1328-43ad-982f-7aecb1ee2008@onelake.dfs.fabric.microsoft.com/900594c3-14cd-4919-8406-174ac04587bb/Files/"

category = spark.read.option("header", "true").csv(path + "category")
customer = spark.read.option("header", "true").csv(path + "customer")
department = spark.read.option("header", "true").csv(path + "department")
product = spark.read.option("header", "true").csv(path + "product")


StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 25, Finished, Available)

In [24]:
from pyspark.sql.functions import current_timestamp, lit, row_number,col
from pyspark.sql.window import Window
from functools import reduce
from pyspark.sql import functions as F

StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 26, Finished, Available)

# department

In [25]:
# Set start_date
department = department.withColumn("start_date", F.current_timestamp())




# Define a window specification
window_spec_key = Window.orderBy("department_id")  
# Add a new column 'product_key' with consecutive IDs starting from 1
department = department.withColumn("department_key", row_number().over(window_spec_key))



# Define window specification
windowSpec = Window.partitionBy("department_id").orderBy("start_date")

# Set end_date
department = department.withColumn("end_date", F.to_timestamp(F.lit("9999-12-31")))


department = department.withColumn("is_valid", lit(True))



StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 27, Finished, Available)

In [26]:
department = department.select("department_key" , "department_id",  "department_name", "start_date", "end_date", "is_valid")

StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 28, Finished, Available)

In [27]:
department.createOrReplaceTempView("department")

StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 29, Finished, Available)

# category

In [28]:
# Set start_date
category = category.withColumn("start_date", F.current_timestamp())




# Define a window specification
window_spec_key = Window.orderBy("category_id")  
# Add a new column 'product_key' with consecutive IDs starting from 1
category = category.withColumn("category_key", row_number().over(window_spec_key))



# Define window specification
windowSpec = Window.partitionBy("category_id").orderBy("start_date")

# Set end_date
category = category.withColumn("end_date", F.to_timestamp(F.lit("9999-12-31")))


category = category.withColumn("is_valid", lit(True))

category.createOrReplaceTempView("category")

StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 30, Finished, Available)

In [29]:
category = spark.sql("""
SELECT c.category_key, c.category_id, c.category_name, d.department_key, c.start_date, c.end_date,c.is_valid

FROM category c JOIN department d ON c.department_id = d.department_id


""")

StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 31, Finished, Available)

In [33]:
category = category.select("category_key", "category_id", "category_name", "department_key", "start_date", "end_date","is_valid")
category.createOrReplaceTempView("category")

StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 35, Finished, Available)

# dim_product

In [31]:
# Set start_date
product = product.withColumn("start_date", F.current_timestamp())




# Define a window specification
window_spec_key = Window.orderBy("product_card_id")  
# Add a new column 'product_key' with consecutive IDs starting from 1
product = product.withColumn("product_key", row_number().over(window_spec_key))



# Define window specification
windowSpec = Window.partitionBy("product_card_id").orderBy("start_date")

# Set end_date
product = product.withColumn("end_date",F.to_timestamp(F.lit("9999-12-31")))


product = product.withColumn("is_valid", lit(True))


product.createOrReplaceTempView("dim_product")


StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 33, Finished, Available)

In [37]:
product = spark.sql("""
SELECT dp.product_key, dp.product_card_id, dp.product_name, dp.product_price, dp.product_status, c.category_key, dp.start_date, dp.end_date, dp.is_valid
FROM dim_product dp JOIN category c 
ON dp.category_id = c.category_id


""")

StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 39, Finished, Available)

In [39]:
product = product.select("product_key", "product_card_id" , "product_name" , "product_price" , "product_status", "category_key", "start_date", "end_date" , "is_valid")

StatementMeta(, 888cce71-408a-4b14-b4bf-09e0b057a71e, 41, Finished, Available)

# dim_customer

In [24]:
# Set start_date
customer = customer.withColumn("start_date", F.current_timestamp())




# Define a window specification
window_spec_key = Window.orderBy("customer_id")  
# Add a new column 'product_key' with consecutive IDs starting from 1
customer = customer.withColumn("customer_key", row_number().over(window_spec_key))



# Define window specification
windowSpec = Window.partitionBy("customer_id").orderBy("start_date")

# Set end_date
customer = customer.withColumn("end_date", F.to_timestamp(F.lit("9999-12-31")))


customer = customer.withColumn("is_valid", lit(True))



StatementMeta(, d5149b37-a0df-43dd-a967-1849e7b644f4, 26, Finished, Available)

In [25]:
customer = customer.select("customer_key","customer_id", "customer_segment", "customer_fullname", "start_date","end_date", "is_valid")
# display(customer)

StatementMeta(, d5149b37-a0df-43dd-a967-1849e7b644f4, 27, Finished, Available)

SynapseWidget(Synapse.DataFrame, eaf1b0b3-550e-4db1-9de3-428532bd0b08)

In [26]:
customer.createOrReplaceTempView("customer")


StatementMeta(, d5149b37-a0df-43dd-a967-1849e7b644f4, 28, Finished, Available)

# 

# Load data in to gold dim

## dim_customer

In [34]:
customer.write.mode("append").format("delta").saveAsTable("dim_customer")

StatementMeta(, d5149b37-a0df-43dd-a967-1849e7b644f4, 36, Finished, Available)

## department


In [ ]:
department.write.mode("append").format("delta").saveAsTable("department")

## category

In [ ]:
category.write.mode("append").format("delta").saveAsTable("category")

## product

In [ ]:
product.write.mode("append").format("delta").saveAsTable("dim_product")